In [1]:
import os
import datajoint as dj
import pathlib

In [3]:
from ephys import reference, acquisition, tracking, behavior, ephys, analysis
mlims = dj.create_virtual_module('mlims', dj.config['custom'].get('mlims.database'))

Connecting thinh@datajoint.it.ntnu.no:3306


# The Ephys Pipeline

This notebook presents the overall structure of the ephys pipeline, including table organization and the accompanied workflow.

# Overview of Session

### The following figure demonstrates the structure of an electrophysiology experiment

<img src="../images/Ephys_Session_Structure.png" alt="drawing" width="700"/>

In one ***Session***, there can be:
+ multiple trackings (in time)
+ multiple recordings (in time)
    + one recording per probe
+ multiple probes (different insertion sites)
+ multiple tasks

Clusterings can be performed on:
+ Multiple concatenated sessions (although in most cases, just on one session)
+ Different time window(s) over the selected session(s)
+ Yielding units for this ***clustering***

For each ***Recording***, ***UnitSpikeTimes*** can be extracted, per ***Unit*** with spike times from this recording

### Following the above experiment organization, the diagram below shows the design of the session related part of the ephys pipeline

![session overview](../images/session_overview_erd.png)

An experiment ***Session*** is uniquely defined by the *animal* and the *datetime* of the experiment

In [9]:
acquisition.Session()

animal_id,datasource_id,session_time start time of this session,session_note
40bfa03a7a7018cc,0,2019-09-17 19:43:06,=BLOB=
724240a0a756020f,0,2018-04-24 10:25:23,=BLOB=
96129eb1563c0b0d,0,2018-06-21 12:59:28,=BLOB=
9d6b85b1e899eb94,0,2014-09-09 12:45:30,=BLOB=
bbd19fe400fac5a6,0,2019-09-10 15:34:57,=BLOB=
d2dc54a46075c5f3,0,2019-09-02 10:03:29,=BLOB=
d2dc54a46075c5f3,0,2019-09-04 10:06:12,=BLOB=


In each ***Session***, a ***Recording*** is uniquely defined by the *datetime* of the recording and the *probe*

The ***Recording*** also contains information about:
+ recording system (e.g. Axona, Neralynx, Neuropixels, Neurologger)
+ electrode configuration (e.g. subset of neuropixels sites, tetrode number)
+ time synchronization 

![recording](../images/recording_detail_erd.png)

In [10]:
acquisition.Recording()

animal_id,datasource_id,session_time start time of this session,insertion_time When this probe was inserted,recording_time start time of this recording,recording_system,"probe_model nick name, or other user-friendly model name of this probe",electrode_config_id hash of the group and group_member (ensure uniqueness),recording_order the ordering of this recording in this session,recording_duration (s) duration of this recording,recording_name name of this recording (e.g. 27032019laserSess1)
40bfa03a7a7018cc,0,2019-09-17 19:43:06,2019-09-11 14:08:45,2019-09-17 19:43:06,neuropixels,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,0,7667.25,npx.imec
40bfa03a7a7018cc,0,2019-09-17 19:43:06,2019-09-11 14:09:22,2019-09-17 19:43:06,neuropixels,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,0,7667.45,npx.imec
724240a0a756020f,0,2018-04-24 10:25:23,2018-03-15 12:00:00,2018-04-24 10:25:23,neurologger,8-tetrode-array,00784804f372476162c7d90e94d8dc3d,0,1800.0,s1
724240a0a756020f,0,2018-04-24 10:25:23,2018-03-15 12:10:00,2018-04-24 10:25:23,neurologger,8-tetrode-array,00784804f372476162c7d90e94d8dc3d,0,1800.0,s1
96129eb1563c0b0d,0,2018-06-21 12:59:28,2018-06-05 10:51:30,2018-06-21 12:59:28,neurologger,8-tetrode-array,00784804f372476162c7d90e94d8dc3d,0,4314.0,s1
9d6b85b1e899eb94,0,2014-09-09 12:45:30,2015-01-01 00:01:00,2014-09-09 12:45:30,axona,4-tetrode-array,222cc704a75a2aea7777cc0899d809fd,0,2406.0,09091403
9d6b85b1e899eb94,0,2014-09-09 12:45:30,2015-01-01 00:02:00,2014-09-09 12:45:30,axona,4-tetrode-array,e1832bbcce7a2aa5fd12f97d7448f9be,0,2406.0,09091403


# Organization of Clustering Data

Clusterings can be performed on:
+ Multiple concatenated sessions (although in most cases, just on one session)
+ Different time window(s) over the selected session(s)
+ Yielding units for this ***clustering***

![](../images/clustering_erd.png)

***ClusteringSetup*** is for users to specify:
+ the ***ClusterSessionGroup*** and ***ClusterTimeWindows***
+ the probe this clustering is performed on - ***ProbeInsertion***
+ the parameters to perform the clustering - ***ClusterParam***
+ the directory to output this clustering results 
+ the electrode configuration information

Note: all of this is taken care of in the ingestion routine (more on that later)

***Clustering*** - a processing step to either trigger the clustering or perform ingestion on performed clustering

***CuratedClustering*** - insert a new curation:
+ the curator
+ the time of curation
+ the directory of the new curation results

In [21]:
ephys.ClusteringSetup.describe();

-> acquisition.ProbeInsertion
-> acquisition.ClusterTimeWindows
-> reference.ClusterParam
-> reference.ElectrodeConfig
-> reference.SyncMethod
---
-> reference.Repository
clustering_output_dir : varchar(128)                 
routine="trigger"    : enum('trigger','import')     
is_synced=1          : tinyint                      # is the clustering output synced to master? - yes if triggered by DataJoint



In [15]:
ephys.CuratedClustering()

animal_id,datasource_id,insertion_time When this probe was inserted,group_name,time_window_id or maybe a hash of all the part windows,cluster_method name of clustering method,"cluster_param_name some unique name, or maybe hash of the dict of the param (ordered)","probe_model nick name, or other user-friendly model name of this probe",electrode_config_id hash of the group and group_member (ensure uniqueness),sync_method sync method name,curation_timestamps ensure complete datetime here to prevent collision (in the rare case of multiple people curate at the exact same datetime),curator,curation_notes,"repository e.g. 'Local', 'Network'",curation_output_dir
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,valentno,,network,neuropixel/shared_data/Roger/2019-09-17_19-41-14/probe_1/ks2.1_01
40bfa03a7a7018cc,0,2019-09-11 14:09:22,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,valentno,,network,neuropixel/shared_data/Roger/2019-09-17_19-41-14/probe_2/ks2.1_01
724240a0a756020f,0,2018-03-15 12:00:00,det_0d7c5dd06d63c5f14395b914f7f803e7,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,8-tetrode-array,00784804f372476162c7d90e94d8dc3d,presync,2019-08-30 13:14:00,torgeirw,,network,sampledata/deuteron/Torgeir/24321/20180424_01_light/probe_1/ks2.1_01
724240a0a756020f,0,2018-03-15 12:10:00,det_0d7c5dd06d63c5f14395b914f7f803e7,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,8-tetrode-array,00784804f372476162c7d90e94d8dc3d,presync,2019-08-30 13:14:00,torgeirw,,network,sampledata/deuteron/Torgeir/24321/20180424_01_light/probe_2/ks2.1_01
96129eb1563c0b0d,0,2018-06-05 10:51:30,det_67ea7668bb2acfeaa3910a4c0e71fb76,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,8-tetrode-array,00784804f372476162c7d90e94d8dc3d,presync,2019-09-16 11:03:40,valentno,,network,valentno/Data/Ratvissant/20180621/1/Processed/Kilosort
bbd19fe400fac5a6,0,2019-09-16 16:47:17,det_a479a5980e5b604b59c0a1216e251a7f,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-16 11:03:40,valentno,,network,neuropixel/vnormand/Ratounette/2019-09-10_15-32-29/probe_1/ks2.1_01_2265-Inf
bbd19fe400fac5a6,0,2019-09-16 16:47:17,det_a479a5980e5b604b59c0a1216e251a7f,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-16 17:27:44,valentno,test,network,neuropixel/vnormand/Ratounette/2019-09-10_15-32-29/probe_1/ks2.1_01_2265-Inf_new


### A more detailed look to the ***Clustering*** pipeline

![](../images/clustering_detail_erd.png)

### With flexibility comes complexity

<img src="../images/Clustering_ChoiceMatrix.png" alt="drawing" width="450"/>

# Toward the results - Spikes and Tracking

![](../images/spike_track_erd.png)

In [25]:
ephys.UnitSpikeTimes()

animal_id,datasource_id,insertion_time When this probe was inserted,group_name,time_window_id or maybe a hash of all the part windows,cluster_method name of clustering method,"cluster_param_name some unique name, or maybe hash of the dict of the param (ordered)","probe_model nick name, or other user-friendly model name of this probe",electrode_config_id hash of the group and group_member (ensure uniqueness),sync_method sync method name,curation_timestamps ensure complete datetime here to prevent collision (in the rare case of multiple people curate at the exact same datetime),unit,session_time start time of this session,recording_time start time of this recording,mean_firing_rate mean firing rate,spike_counts how many spikes in this recording of this unit,"unit_spike_times (s) spike times of this unit, relative to the start of the session this recording belongs to"
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,0,2019-09-17 19:43:06,2019-09-17 19:43:06,2.12436,16288,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,1,2019-09-17 19:43:06,2019-09-17 19:43:06,4.88728,37472,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,2,2019-09-17 19:43:06,2019-09-17 19:43:06,3.87792,29733,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,3,2019-09-17 19:43:06,2019-09-17 19:43:06,1.21243,9296,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,4,2019-09-17 19:43:06,2019-09-17 19:43:06,0.283544,2174,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,5,2019-09-17 19:43:06,2019-09-17 19:43:06,0.618866,4745,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,6,2019-09-17 19:43:06,2019-09-17 19:43:06,0.0164335,126,=BLOB=


***UnitSpikeTimes*** and ***Tracking*** data can be matched together - resulting in the ***SpikesTracking***:
> The spike times and tracking data (e.g. position, speed, head angle, etc.) associated with each spike, per unit

And such ***SpikesTracking*** can then be further narrowed down to the task, in ***TaskSpikesTracking***

In [26]:
ephys.SpikesTracking()

animal_id,datasource_id,insertion_time When this probe was inserted,group_name,time_window_id or maybe a hash of all the part windows,cluster_method name of clustering method,"cluster_param_name some unique name, or maybe hash of the dict of the param (ordered)","probe_model nick name, or other user-friendly model name of this probe",electrode_config_id hash of the group and group_member (ensure uniqueness),sync_method sync method name,curation_timestamps ensure complete datetime here to prevent collision (in the rare case of multiple people curate at the exact same datetime),unit,session_time start time of this session,recording_time start time of this recording,tracking_time start time of this tracking recording,tracking_processing_method,spike_times (s) repeat of SyncedSpikeTimes (for convenient queries),speed (cm/s) tracked speed at each spike times (time with respect to the start of session),x_pos (cm) tracked x-pos at each spike times (time with respect to the start of session),y_pos (cm) tracked y-pos at each spike times (time with respect to the start of session),z_pos (cm) tracked z-pos at each spike times (time with respect to the start of session),head_yaw (degree) tracked head yaw at each spike times (time with respect to the start of session),head_pitch (degree) tracked head pitch at each spike times (time with respect to the start of session),head_roll (degree) tracked head roll at each spike times (time with respect to the start of session),angular_speed (degree/s) tracked head angular speed at each spike times (time with respect to the start of session)
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,3,2019-09-17 19:43:06,2019-09-17 19:43:06,2019-09-17 19:42:39,3d_optitrack,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,4,2019-09-17 19:43:06,2019-09-17 19:43:06,2019-09-17 19:42:39,3d_optitrack,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,8,2019-09-17 19:43:06,2019-09-17 19:43:06,2019-09-17 19:42:39,3d_optitrack,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,9,2019-09-17 19:43:06,2019-09-17 19:43:06,2019-09-17 19:42:39,3d_optitrack,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,10,2019-09-17 19:43:06,2019-09-17 19:43:06,2019-09-17 19:42:39,3d_optitrack,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,11,2019-09-17 19:43:06,2019-09-17 19:43:06,2019-09-17 19:42:39,3d_optitrack,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,12,2019-09-17 19:43:06,2019-09-17 19:43:06,2019-

In [27]:
analysis.TaskSpikesTracking()

animal_id,datasource_id,session_time start time of this session,task_type,task_start (s) timestamp of task onset with respect to the start of the session,unit unit id,spike_tracking_hash hash of the SpikesTracking primary_key,cell_selection_params_name user-friendly name,spike_times (s) task-related spike times (time with respect to the start of the task),speed (cm/s) task-related speed at each spike times,x_pos (cm) task-related x-pos at each spike times,y_pos (cm) task-related y-pos at each spike times,z_pos (cm) task-related z-pos at each spike times,head_yaw (degree) task-related head yaw (left/right angle) at each spike times,head_pitch (degree) task-related head pitch (up/down) at each spike times,head_roll (degree) task-related head roll (clockwise/anticlockwise) at each spike times,angular_speed (degree/s) task-related head angular speed at each spike times
724240a0a756020f,0,2018-04-24 10:25:23,OpenField,0.00,0,2302a64000a8b5c6a347d228cede17b5,default,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
724240a0a756020f,0,2018-04-24 10:25:23,OpenField,0.00,1,8f26e58aa05acd610b65e908a907f038,default,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
724240a0a756020f,0,2018-04-24 10:25:23,OpenField,0.00,4,3b9eefe18801652fd5cf5901fb6a97f8,default,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
724240a0a756020f,0,2018-04-24 10:25:23,OpenField,0.00,6,209db0c7f1b4ba016ce05d703333d391,default,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
724240a0a756020f,0,2018-04-24 10:25:23,OpenField,0.00,12,6e812d06eb6de96d9517a94a9735f56f,default,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
724240a0a756020f,0,2018-04-24 10:25:23,OpenField,0.00,13,e29ae5f77cccb7d0acf5d5d020e110f1,default,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
724240a0a756020f,0,2018-04-24 10:25:23,OpenField,0.00,14,d42933ad6edea511b74abb397cfa8239,default,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
